In [1]:
import datajoint as dj

In [2]:
dj.__version__

'0.12.5'

In [3]:
# datajoint configuration
dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
# we will go into this attribute later
dj.config['fetch_format'] = 'frame'

In [4]:
# connect to the database
# testing your connection
connection = dj.conn()
connection

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [5]:
dj.config.save_local()

In [6]:
# I haven't done this
#dj.config.save_global() 

In [7]:
dj.set_password('simple')

Password updated.
Update local setting? [yes, no]: 
Update local setting? [yes, no]: yes
Saved settings in dj_local_conf.json


In [8]:
schema = dj.schema('homework1')

In [10]:
@schema
class FlySubject(dj.Manual):
    definition = """
    # some fly table
    subject_id : int  # id for fly subject
    ---
    age : float # age of fly in days
    sex = 'U': enum('F', 'M', 'U')  # sex of fly
    comments = null : varchar(4000)
    """

In [11]:
fly_subject = FlySubject()
fly_subject

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments


In [14]:
# using a tuple
fly_subject.insert1((0, 3.4, 'M', 'This subject was particularly eager to eat my cake and have it too.'))

# using a dictionary
fly_subject.insert1({'subject_id': 1, 'age':4.5})



DuplicateError: ("Duplicate entry '0' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [15]:
skip_duplicates=True

In [16]:
fly_subject

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments
0,3.4,M,This subject was particularly eager to eat my cake and have it too.
1,4.5,U,None


In [17]:
fly_subject.insert([
    {'subject_id': 2, 'age':4.5, 'sex': 'M', 'comments':None}, 
    {'subject_id': 3, 'age':7.5, 'sex': 'F', 'comments':'some comments'}, 
    {'subject_id': 4, 'age':9.5, 'sex': 'F', 'comments':'other comments'}, 
])

In [18]:
fly_subject

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments
0,3.4,M,This subject was particularly eager to eat my cake and have it too.
1,4.5,U,None
2,4.5,M,None
3,7.5,F,some comments
4,9.5,F,other comments


In [19]:
fly_subject & 'age > 5'

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments
3,7.5,F,some comments
4,9.5,F,other comments


In [20]:
fly_subject & 'age > 5' & 'sex = "M"'

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments


In [21]:
fly_subject & {'sex': 'M', 'age': 4.5}

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments
2,4.5,M,None


In [22]:
fly_subject - {'sex':'M'}

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments
1,4.5,U,None
3,7.5,F,some comments
4,9.5,F,other comments


In [23]:
fly_subject.proj() # only the primary keys

subject_id id for fly subject
0
1
2
3
4


In [24]:
fly_subject.proj('sex') # primary keys and sex column

subject_id id for fly subject,sex sex of fly
0,M
1,U
2,M
3,F
4,F


In [25]:
fly_subject.proj('sex', 'age') # primary keys and sex, age column

subject_id id for fly subject,age age of fly in days,sex sex of fly
0,3.4,M
1,4.5,U
2,4.5,M
3,7.5,F
4,9.5,F


In [26]:
fly_subject.proj(age_in_days='age') # rename age into age_in_days

subject_id id for fly subject,age_in_days age of fly in days
0,3.4
1,4.5
2,4.5
3,7.5
4,9.5


In [27]:
(fly_subject & {'sex':'M'}).proj() # restrict then project

subject_id id for fly subject
0
2


In [28]:
(fly_subject.proj() & {'sex':'M'}) # project then restrict
# since we projected first the sex column dropped before we restricted by it

subject_id id for fly subject
0
1
2
3
4


In [29]:
fly_subject.fetch() # the index is set to the primary keys

,age,sex,comments
subject_id,,,
0,3.4,M,This subject was particularly eager to eat my ...
1,4.5,U,None
2,4.5,M,None
3,7.5,F,some comments
4,9.5,F,other comments


In [30]:
fly_subject.fetch(as_dict=True)

[{'subject_id': 0,
  'age': 3.4,
  'sex': 'M',
  'comments': 'This subject was particularly eager to eat my cake and have it too.'},
 {'subject_id': 1, 'age': 4.5, 'sex': 'U', 'comments': None},
 {'subject_id': 2, 'age': 4.5, 'sex': 'M', 'comments': None},
 {'subject_id': 3, 'age': 7.5, 'sex': 'F', 'comments': 'some comments'},
 {'subject_id': 4, 'age': 9.5, 'sex': 'F', 'comments': 'other comments'}]

In [31]:
fly_subject.fetch(format='array') # here we have it as a record array

array([(0, 3.4, 'M', 'This subject was particularly eager to eat my cake and have it too.'),
       (1, 4.5, 'U', None), (2, 4.5, 'M', None),
       (3, 7.5, 'F', 'some comments'), (4, 9.5, 'F', 'other comments')],
      dtype=[('subject_id', '<i8'), ('age', '<f8'), ('sex', 'O'), ('comments', 'O')])

In [32]:
table = (fly_subject - {'sex':'M'}).fetch() # here i am also assigning the fetch table to the table variable
table  # look no male flies

,age,sex,comments
subject_id,,,
1,4.5,U,None
3,7.5,F,some comments
4,9.5,F,other comments


In [33]:
(fly_subject & {'sex':'M'}).proj().fetch() # fetching a projection

""
subject_id
0
2


In [34]:
(fly_subject & {'sex':'M', 'age':4.5}).fetch1() # fetching a single entry

{'subject_id': 2, 'age': 4.5, 'sex': 'M', 'comments': None}

In [35]:
(fly_subject).fetch1()

DataJointError: fetch1 should only be used for relations with exactly one tuple

In [36]:
(fly_subject & {'sex':'M'}).delete()

About to delete
`homework1`.`fly_subject`: 2 items
Proceed? [yes, No]: yes
Commited.


In [37]:
fly_subject

subject_id id for fly subject,age age of fly in days,sex sex of fly,comments
1,4.5,U,None
3,7.5,F,some comments
4,9.5,F,other comments


In [38]:
fly_subject.drop()

`homework1`.`fly_subject` (3 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [39]:
schema.drop()

Proceed to delete entire schema `homework1`? [yes, No]: yes


In [40]:
import my_database as db

In [42]:
# you can take a look at the schema and all connections with datajoint.ERD
dj.ERD(db.schema)

FileNotFoundError: [Errno 2] "dot" not found in path.

In [43]:
# this is the definition for RecordingSession
print(db.RecordingSession.definition)


    # record them sessions
    -> FlySubject
    recording_id : int 
    ---
    -> Stimulus
    experimenter : varchar(127)
    recording_quality : enum('good', 'bad', 'ugly')
    comments = null : varchar(4000)
    
